### Ensemble trees for kaviar
* duplicate positons are removed
* focus on cgi vs both, limit by ahmad good regions
* focus on high freq vars?

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
%matplotlib inline

def write_feature_importance(
    X, indices, cols, importances, std,
):
    for f in range(X.shape[1]):
        ls = (
            cols[indices[f]],
            importances[indices[f]],
            std[indices[f]],
        )
        print("%s\t%f\t%f" % ls)

In [2]:
kaviar_mat_file = '/mnt/isilon/cbmi/variome/perry/brian/explore-cgi/data/interim/kaviar.mat'
dat = pd.read_csv(kaviar_mat_file, sep='\t')

/opt/conda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# limit to not rare
#dat = dat[dat.af<2**-14]
features = [x for x in dat.columns if not x in ('kaviar_status', 'var_type', 'chrom', 'pos', 'ref', 'alt', 'af')]
dat = dat.drop_duplicates(subset=['chrom', 'pos'])

In [4]:
dat_cgi = dat[((dat['ahmad_status'] == 1) | (dat['ahmad_status'] == 0)) & ((dat.kaviar_status=='cgi') | (dat.kaviar_status=='both'))]
dat_cgi.loc[:, 'y'] = dat_cgi.apply(lambda row: 1 if row['kaviar_status'] == 'both' else 0, axis=1)
dat_cgi.groupby('y').size()

/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


y
0    142095
1    177058
dtype: int64

In [5]:
#dat_cgi = dat[((dat['ahmad_status'] == 1) | (dat['ahmad_status'] == 0)) & ((dat.kaviar_status=='cgi') | (dat.kaviar_status=='both'))]
#dat_cgi.loc[:, 'y'] = dat_cgi.apply(lambda row: 1 if row['kaviar_status'] == 'both' else 0, axis=1)

pos_cgi = dat_cgi[dat_cgi.y==1].sample(141900)
neg_cgi = dat_cgi[dat_cgi.y==0].sample(141900)
dat_cgi = pd.concat([pos_cgi, neg_cgi])

X = dat_cgi[features].values
X = StandardScaler().fit_transform(X)
y = dat_cgi['y'].values

rfc = RandomForestClassifier(n_estimators=200)
clf = svm.SVC(kernel='poly', C=1)
clf_tree = tree.DecisionTreeClassifier(max_depth=2)
scores = cross_val_score(rfc, X, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

dat_cgi.groupby('kaviar_status').size()

/opt/conda/lib/python3.4/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Accuracy: 0.63 (+/- 0.01)


kaviar_status
both    141900
cgi     141900
dtype: int64

In [6]:
cols = features
forest = ExtraTreesClassifier(n_estimators=250, random_state=0)
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]
write_feature_importance(
    X, indices, cols, importances, std
)

AllRepeats_gt95percidentity_slop5	0.185566	0.174284
rmsk	0.173931	0.170296
1kg	0.136593	0.123463
20120824_combined_mask	0.126603	0.127295
AllRepeats_lt51bp_gt95identity_merged	0.108822	0.145348
notinAllRepeats_gt95percidentity_slop5	0.049088	0.033847
ahmad_status	0.040571	0.030480
simpleRepeat	0.031999	0.044069
notinrefseq_union_cds.sort	0.028834	0.027132
human_g1k_v37_l100_gclt30orgt55_slop50	0.022037	0.017715
human_g1k_v37_l100_gc30to55_slop50	0.020261	0.024953
dgv.short	0.018502	0.004152
dgv	0.008903	0.003915
GRCh37GenomicSuperDup.sorted	0.008308	0.001846
hg19_self_chain_split.sort	0.006740	0.001811
hg19_self_chain_split_both	0.006166	0.002118
lowmappabilityall	0.005970	0.010250
siren_similarRegions_dist1	0.005782	0.009207
notinsegdupall	0.005133	0.006219
notinlowmappabilityall	0.004367	0.008522
segdupall	0.003592	0.004521
BadPromoters_gb-2013-14-5-r51-s1	0.002139	0.001139
blackTerry	0.000095	0.000037
hg19.blacklist	0.000000	0.000000


In [7]:
pca = PCA()
X_r = pca.fit(X).transform(X)

lda = LinearDiscriminantAnalysis()
X_r2 = lda.fit(X, y).transform(X)

print(pca.explained_variance_ratio_)

plt.figure()
colors = ['navy', 'darkorange']
lw = 2

for color, i, status in zip(colors, [0, 1], kaviar_status):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of Kaviar dataset')

plt.figure()
for color, i, status in zip(colors, [0, 1], kaviar_status):
    plt.scatter(X_r2[y == i, 0], X_r2[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA of Kaviar dataset')

plt.show()

[  2.25492629e-01   1.36837362e-01   1.23607210e-01   7.31799251e-02
   6.33055482e-02   5.21172082e-02   4.72740610e-02   4.34027213e-02
   4.29930228e-02   3.67048663e-02   3.35219027e-02   2.76114483e-02
   2.55602326e-02   2.35901640e-02   1.30564695e-02   1.15042656e-02
   8.84543094e-03   4.33300985e-03   2.90960709e-03   2.63653486e-03
   1.17444783e-03   3.41933493e-04   3.44046136e-33   6.45349833e-36]


/opt/conda/lib/python3.4/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


NameError: name 'kaviar_status' is not defined